# Import

In [ ]:
import time
import matplotlib
import matplotlib.pyplot as plt
from keras.models import model_from_json
from keras.optimizers import Adam
from keras import backend as K
import h5py
import keras
import numpy
import os
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import cv2
from PIL import Image

ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)
from Model import multi_resolution
from utils import get_err,hand_utils
ROOT_DIR = os.path.abspath("../Hier_Estimator")
sys.path.append(ROOT_DIR)
import trainer

# Init

In [ ]:
setname='mega'

_EPSILON = 10e-8

K.set_image_data_format('channels_last')  # TF dimension ordering in this code
num_kern=[32,64,96]

lr=0.00003
# best lr = 0.0001

# version = 'palm_s0_rot_scale_ker%d_lr%f'%(num_kern[0],lr)

data_dir = '../data'
source_dir= os.path.join(data_dir, 'mega')
save_dir= os.path.join(data_dir, 'mega')
img_dir = os.path.join(source_dir, 'test_images')
palm_idx =[0,1,5,9,13,17]
batch_size=128


# Load Data

In [ ]:
dataset = 'test'

f = h5py.File('%s/source/%s_crop_norm_vassi.h5'%(save_dir,dataset), 'r')
print('keys=', list(f.keys()))
test_x0 = f['img0'][...]
test_x1 = f['img1'][...]
test_x2 = f['img2'][...]
test_y= f['uvd_norm_gt'][...][:,palm_idx,:].reshape(-1,len(palm_idx)*3)
uvd_gt = f['uvd_gt'][...][:,palm_idx,:]
# xyz_gt = f['xyz_gt'][...][:,palm_idx,:]
bbsize = f['bbsize']
new_file_names = f['new_file_names'][...]
uvd_hand_centre = f['uvd_hand_centre']
f.close()
print(dataset,' loaded',test_x0.shape,test_y.shape)
# data = numpy.expand_dims(test_x0,axis=-1),numpy.expand_dims(test_x1,axis=-1),numpy.expand_dims(test_x2,axis=-1),test_y

In [ ]:
idx=0
cur_frame = new_file_names[idx]
depth = Image.open("%s/%s.png"%(img_dir,cur_frame))
depth = numpy.asarray(depth, dtype='uint16')

imgcopy=depth.copy()
min = imgcopy.min()
max = imgcopy.max()
#scale to 0 - 255
imgcopy = (imgcopy - min) / (max - min) * 255. 
imgcopy = imgcopy.astype('uint8')
imgcopy = cv2.cvtColor(imgcopy, cv2.COLOR_GRAY2BGR)

#visualize annotation
for j in range(uvd_gt.shape[1]):
    cv2.circle(imgcopy,(int(uvd_gt[idx,j,0]),int(uvd_gt[idx,j,1])), int(3000.0/numpy.mean(uvd_gt[idx,j,2])), (0, 255, 0), -1)
    
print(cur_frame)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(imgcopy)



In [ ]:
fig, ax = plt.subplots(1,3,figsize=(18, 16))
ax[0].imshow(test_x0[idx])
ax[0].set_title('96')
ax[1].imshow(test_x1[idx])
ax[1].set_title('48')
ax[2].imshow(test_x2[idx])
ax[2].set_title('24')

# Augmentation

In [ ]:
def augment_data_3d_mega_rot_scale(r0,r1,r2,gr_uvd):
    new_r0=r0[:,:,:,0].copy()
    new_r1=r1[:,:,:,0].copy()
    new_r2=r2[:,:,:,0].copy()

    new_gr_uvd =gr_uvd.copy().reshape(gr_uvd.shape[0],-1,3)

    num_frame=gr_uvd.shape[0]
    num_jnt=new_gr_uvd.shape[1]

    img_gr_uv =new_gr_uvd[:,:,:2]*96+48

    rot = numpy.random.uniform(low=-180,high=180,size=num_frame)
    scale_factor = numpy.random.normal(loc=1,scale=0.05,size=num_frame)

    # for i in range(0,gr_uvd.shape[0],1):
    for i in numpy.random.randint(0,num_frame,int(num_frame*0.5)):
        """2d translation, rotation and scale"""
        # print(center_x[i],center_y[i],rot[i],scale_factor[i])
        M = cv2.getRotationMatrix2D((48,48),rot[i],scale_factor[i])
        new_r0[i] = cv2.warpAffine(new_r0[i],M,(96,96),borderValue=1)

        for j in range(num_jnt):
            tmp=numpy.dot(M,numpy.array([img_gr_uv[i,j,0],img_gr_uv[i,j,1],1]))
            new_gr_uvd[i,j,0:2] = (tmp-48)/96
        M = cv2.getRotationMatrix2D((24,24),rot[i],scale_factor[i])
        new_r1[i] = cv2.warpAffine(new_r1[i],M,(48,48),borderValue=1)
        M = cv2.getRotationMatrix2D((12,12),rot[i],scale_factor[i])
        new_r2[i] = cv2.warpAffine(new_r2[i],M,(24,24),borderValue=1)

    return numpy.expand_dims(new_r0,axis=-1),numpy.expand_dims(new_r1,axis=-1),\
           numpy.expand_dims(new_r2,axis=-1),new_gr_uvd.reshape(new_gr_uvd.shape[0],new_gr_uvd.shape[1]*new_gr_uvd.shape[2])

train_img0 = numpy.expand_dims(test_x0, axis=-1)
train_img1 = numpy.expand_dims(test_x1, axis=-1)
train_img2 = numpy.expand_dims(test_x2, axis=-1)
train_target = test_y

batch_idx = [0,1]
x0,x1,x2,y= augment_data_3d_mega_rot_scale(train_img0[batch_idx],train_img1[batch_idx],train_img2[batch_idx],
                                                                train_target[batch_idx])
fig, ax = plt.subplots(1,3,figsize=(18, 16))
ax[0].imshow(numpy.squeeze(x0[0]))
ax[0].set_title('96')
ax[1].imshow(numpy.squeeze(x1[0]))
ax[1].set_title('48')
ax[2].imshow(numpy.squeeze(x2[0]))
ax[2].set_title('24')